In [1]:
# append the module root to Python path
import sys

sys.path.append("../src")

In [2]:
from mygo.datasets import MCTSDataset
from torch.utils.data import DataLoader
import torch
from torch import nn

In [3]:
torch.manual_seed(25565)  # for reproducibility

In [4]:
train_data = MCTSDataset(
    root="../data",
    train=True,
    transform=torch.from_numpy,
    target_transform=torch.from_numpy,
)

test_data = MCTSDataset(
    root="../data",
    train=False,
    transform=torch.from_numpy,
    target_transform=torch.from_numpy,
)

In [5]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

In [6]:
class TinyModel(nn.Module):
    """Tiny model for 9x9 game."""

    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(81, 128)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(128, 81)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)

        return x


model = TinyModel()
print(model)

TinyModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=81, out_features=128, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=128, out_features=81, bias=True)
  (softmax): Softmax(dim=0)
)


In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    batches = len(dataloader)
    model.train()
    for i, (xs, ys) in enumerate(dataloader):
        optimizer.zero_grad()

        pred = model(xs)
        loss = loss_fn(pred, ys.flatten(start_dim=1))

        loss.backward()
        optimizer.step()

        if i % 10 == 9:
            print(f"loss: {loss.item():>7f} [{i + 1:>2d}/{batches:>2d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for xs, ys in dataloader:
            pred = model(xs)
            test_loss += loss_fn(pred, ys.flatten(start_dim=1)).item()
            correct += (
                (pred.argmax(1) == ys.flatten(start_dim=1).argmax(1))
                .type(torch.float)
                .sum()
                .item()
            )

    test_loss /= batches
    accuracy = correct / size * 100

    print(f"\nTest Accuracy: {correct:>.1f}%\nAvg Loss: {test_loss:>7f}\n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters())
epoches = 10

for i in range(epoches):
    print(f"Epoch {i + 1}\n{'-' * 25}")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)

Epoch 1
-------------------------
loss: 4.395415 [10/39]
loss: 4.393528 [20/39]
loss: 4.394171 [30/39]

Test Accuracy: 2.0%
Avg Loss: 4.394043

Epoch 2
-------------------------
loss: 4.392640 [10/39]
loss: 4.395918 [20/39]
loss: 4.393864 [30/39]

Test Accuracy: 3.0%
Avg Loss: 4.394972

Epoch 3
-------------------------
loss: 4.387228 [10/39]
loss: 4.384843 [20/39]
loss: 4.383787 [30/39]

Test Accuracy: 3.0%
Avg Loss: 4.394781

Epoch 4
-------------------------
loss: 4.388971 [10/39]
loss: 4.389266 [20/39]
loss: 4.383156 [30/39]

Test Accuracy: 2.0%
Avg Loss: 4.397072

Epoch 5
-------------------------
loss: 4.389380 [10/39]
loss: 4.387064 [20/39]
loss: 4.384323 [30/39]

Test Accuracy: 0.0%
Avg Loss: 4.398990

Epoch 6
-------------------------
loss: 4.381176 [10/39]
loss: 4.383199 [20/39]
loss: 4.377524 [30/39]

Test Accuracy: 0.0%
Avg Loss: 4.393631

Epoch 7
-------------------------
loss: 4.371175 [10/39]
loss: 4.373052 [20/39]
loss: 4.372859 [30/39]

Test Accuracy: 0.0%
Avg Loss: 4.